<a href="https://colab.research.google.com/github/Tamim-saad/DL_practice/blob/master/How_to_Select_how_many_hidden_layer_and_neurons_in_a_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## How to Select how many hidden layer and neurons in a neural network

In [ ]:

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU

from sklearn.preprocessing import StandardScaler

Using TensorFlow backend.


In [ ]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values



In [ ]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [ ]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]



C:\Users\krish.naik\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\sklearn\preprocessing\_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\krish.naik\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\sklearn\preprocessing\_encoders.py:390: DeprecationWarning: The 'categorical_features' keyword is deprecated in version 0.20 and will be removed in 0.22. You can use the ColumnTransformer instead.
  "use the ColumnTransformer instead.", DeprecationWarning)


In [ ]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) # Note: no activation beyond this point

    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=create_model, verbose=0)

In [ ]:
model

In [ ]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)

This code block is setting up a GridSearchCV object, which is a powerful tool for hyperparameter tuning. It systematically works through multiple combinations of parameter values to find the best performing model. Let's break it down line by line:

layers = [[20], [40, 20], [45, 30, 15]]:

This defines a list named layers. Each inner list represents a different neural network architecture, specifically the number of neurons in each hidden layer. For example:
[20] means one hidden layer with 20 neurons.
[40, 20] means two hidden layers, the first with 40 neurons and the second with 20 neurons.
[45, 30, 15] means three hidden layers with 45, 30, and 15 neurons respectively.
activations = ['sigmoid', 'relu']:

This defines a list of activation functions that will be tested for the hidden layers of the neural network. sigmoid and relu (Rectified Linear Unit) are common choices.
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30]):

This creates a dictionary called param_grid. This dictionary specifies the hyperparameters and their possible values that GridSearchCV will explore:
'layers': The key refers to the layers argument in your create_model function (via the KerasClassifier wrapper). Its values are taken from the layers list you just defined.
'activation': The key refers to the activation argument in your create_model function. Its values are taken from the activations list.
'batch_size': This is a hyperparameter for the training process itself, controlling the number of samples per gradient update. Here, it will test batch sizes of 128 and 256.
'epochs': This is another training hyperparameter, specifying the number of times the entire training dataset will be passed forward and backward through the neural network. In this case, it's set to 30 epochs.
GridSearchCV will try every possible combination of these values (e.g., layers=[20], activation='sigmoid', batch_size=128, epochs=30; then layers=[20], activation='sigmoid', batch_size=256, epochs=30, and so on).
grid = GridSearchCV(estimator=model, param_grid=param_grid):

This line initializes the GridSearchCV object:
estimator=model: This is the machine learning model (or estimator) you want to tune. In your case, it's the KerasClassifier instance you created earlier, which wraps your Keras neural network model.
param_grid=param_grid: This is the dictionary of hyperparameters and their values that GridSearchCV will search through.
When you later call grid.fit(), GridSearchCV will train a separate model for each combination of hyperparameters specified in param_grid, using cross-validation to evaluate their performance and find the best one.

In [ ]:
grid_result = grid.fit(X_train, y_train)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


C:\Users\krish.naik\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


In [ ]:
[grid_result.best_score_,grid_result.best_params_]

[0.8387499995157123,
 {'activation': 'relu',
  'batch_size': 256,
  'epochs': 30,
  'layers': [45, 30, 15]}]

In [ ]:
pred_y = grid.predict(X_test)

In [ ]:
y_pred = (pred_y > 0.5)

In [ ]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)

In [ ]:
cm

array([[1595,    0],
       [ 405,    0]], dtype=int64)

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)

In [ ]:
score

0.7975